# Sparkify Project Workspace
This workspace contains a tiny subset (128MB) of the full dataset available (12GB). Feel free to use this workspace to build your project, or to explore a smaller subset with Spark before deploying your cluster on the cloud. Instructions for setting up your Spark cluster is included in the last lesson of the Extracurricular Spark Course content.

You can follow the steps below to guide your data analysis and model building portion of this project.

In [2]:
# import libraries
import pyspark as spark
from pyspark.sql import SparkSession
import zipfile
import numpy as np
import plotly.graph_objects as go

# unzip compressed data file
zip_ref = zipfile.ZipFile('mini_sparkify_event_data.json.zip', 'r')
zip_ref.extractall()
zip_ref.close()

In [3]:
# create a Spark session
spark = SparkSession.builder \
        .appName("Sparkify") \
        .getOrCreate()


# Load and Clean Dataset
In this workspace, the mini-dataset file is `mini_sparkify_event_data.json`. Load and clean the dataset, checking for invalid or missing data - for example, records without userids or sessionids. 

In [4]:
# load data set
data = spark.read.json("mini_sparkify_event_data.json")
data.show()
data.columns
data.count()

+--------------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+---------------+-------------+---------+--------------------+------+-------------+--------------------+------+
|              artist|     auth|firstName|gender|itemInSession|lastName|   length|level|            location|method|           page| registration|sessionId|                song|status|           ts|           userAgent|userId|
+--------------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+---------------+-------------+---------+--------------------+------+-------------+--------------------+------+
|      Martha Tilston|Logged In|    Colin|     M|           50| Freeman|277.89016| paid|     Bakersfield, CA|   PUT|       NextSong|1538173362000|       29|           Rockpools|   200|1538352117000|Mozilla/5.0 (Wind...|    30|
|    Five Iron Frenzy|Logged In|    Micah|     M|           79|    Long|236.09424| free|Bost

286500

In [19]:
data.createOrReplaceTempView("data")

In [20]:
# clean data
# Try to figure out which columns have missing values, are NULL
#for column in in data.columns

print('Check if Null')
for column in data.columns:
    print(column,spark.sql(f"SELECT * FROM data WHERE {column} IS NULL").count())
print()
print('Check if empty string')
for column in data.columns:
    print(column,spark.sql(f"SELECT * FROM data WHERE {column} = ''").count())

Check if Null
artist 58392
auth 0
firstName 8346
gender 8346
itemInSession 0
lastName 8346
length 58392
level 0
location 8346
method 0
page 0
registration 8346
sessionId 0
song 58392
status 0
ts 0
userAgent 8346
userId 0

Check if empty string
artist 0
auth 0
firstName 0
gender 0
itemInSession 0
lastName 0
length 0
level 0
location 0
method 0
page 0
registration 0
sessionId 0
song 0
status 0
ts 0
userAgent 0
userId 8346


- No missing values for auth, itemInSession, level, method, page, sessionID, status, ts, userID
- but 8346 empty string userIds
We are not going to consider empty string userIds therefore we create a new data frame

In [21]:
data_cleaned = spark.sql(f"SELECT * FROM data WHERE userId != ''")
data_cleaned.createOrReplaceTempView("data_cleaned")

In [22]:
# Number of unique items in each row
for column in data_cleaned.columns:
    print(column,spark.sql(f"SELECT COUNT(DISTINCT({column})) FROM data_cleaned").collect()[0][0])

artist 17655
auth 2
firstName 189
gender 2
itemInSession 1311
lastName 173
length 14865
level 2
location 114
method 2
page 19
registration 225
sessionId 2312
song 58480
status 3
ts 269770
userAgent 56
userId 225


- So 225 unique users, that we have to make predictions
- Lets make a plot 

In [23]:
# Comparison of number of interactions
paid_user_interactions = np.array(data_cleaned[data_cleaned['level']=='paid'].groupBy('userId').count().orderBy('count', ascending=False).collect()).astype(int)
free_user_interactions = np.array(data_cleaned[data_cleaned['level']=='free'].groupBy('userId').count().orderBy('count', ascending=False).collect()).astype(int)

# Comparison number of paied and free users
fig = go.Figure()
fig.add_trace(go.Pie(labels=['free','paid'], values = [free_user_interactions.shape[0], paid_user_interactions.shape[0]]))
fig.update_layout(title_text="Fraction of free and paying users",
                  template='plotly_white')
fig.show()
# Comparison of number of interactions
fig = go.Figure()
fig.add_trace(go.Histogram(x=free_user_interactions[:,1], name = 'free'))
fig.add_trace(go.Histogram(x=paid_user_interactions[:,1], name = 'paid'))
fig.update_xaxes(title_text='Number of interactions')
fig.update_yaxes(title_text='Number of users')
fig.update_layout(title_text="Comparison interactions free and paying users",
                  template='plotly_white')
fig.show()

In [24]:
# Statistics about the number of interactions per session
free_interactions_per_session = np.array(data_cleaned[data_cleaned['level'] == 'free'].groupBy('sessionId').count().collect()).astype(int)
paid_interactions_per_session = np.array(data_cleaned[data_cleaned['level'] == 'paid'].groupBy('sessionId').count().collect()).astype(int)
fig = go.Figure()
fig.add_trace(go.Box(y=free_interactions_per_session[:,1], name = 'Free'))
fig.add_trace(go.Box(y=paid_interactions_per_session[:,1], name = 'Paid'))
fig.update_yaxes(title_text='Number of interactions per session')
fig.update_layout(title_text="Boxplot number of interactions per session",
                  template='plotly_white')
fig.show()

In [26]:
# Count of the interactions with the different pages
free_count_of_pages = np.array(data_cleaned[data_cleaned['level'] == 'free'].groupBy('page').count().orderBy('count', ascending=False).collect())
paid_count_of_pages = np.array(data_cleaned[data_cleaned['level'] == 'paid'].groupBy('page').count().orderBy('count', ascending=False).collect())
fig = go.Figure()
fig.add_trace(go.Bar(x=free_count_of_pages[:,1].astype(float), y=free_count_of_pages[:,0], orientation='h', name='free'))
fig.add_trace(go.Bar(x=paid_count_of_pages[:,1].astype(float), y=paid_count_of_pages[:,0], orientation='h', name='paid'))
fig.update_layout(title_text="Number of page interactions",
                  height=600,
                  template='plotly_white')
fig.show()

In [27]:
# Count of the interactions with the different pages for unique users per page
import pyspark.sql.functions as func
free_count_of_pages = np.array(data_cleaned[data_cleaned['level'] == 'free'].groupby('page').agg(func.countDistinct('userID')).collect())
paid_count_of_pages = np.array(data_cleaned[data_cleaned['level'] == 'paid'].groupby('page').agg(func.countDistinct('userID')).collect())


fig = go.Figure()
fig.add_trace(go.Bar(x=free_count_of_pages[:,1].astype(float), y=free_count_of_pages[:,0], orientation='h', name='unique free users'))
fig.add_trace(go.Bar(x=paid_count_of_pages[:,1].astype(float), y=paid_count_of_pages[:,0], orientation='h', name='unqiue paid users'))
fig.update_layout(title_text="Number of page interactions for unqiue users",
                  height=600,
                  template='plotly_white')
fig.show()


In [106]:
df_cancel_avg = data_cleaned[data_cleaned['page'] == 'Cancellation Confirmation'].groupby('userId').mean().toPandas()
# Plot average item in Session length and time between registration and cancelation
data_cleaned.groupby('page').agg(func.countDistinct('userID')).show()

+--------------------+----------------------+
|                page|count(DISTINCT userID)|
+--------------------+----------------------+
|              Cancel|                    52|
|    Submit Downgrade|                    49|
|         Thumbs Down|                   203|
|                Home|                   223|
|           Downgrade|                   154|
|         Roll Advert|                   207|
|              Logout|                   213|
|       Save Settings|                   132|
|Cancellation Conf...|                    52|
|               About|                   155|
|            Settings|                   195|
|     Add to Playlist|                   215|
|          Add Friend|                   206|
|            NextSong|                   225|
|           Thumbs Up|                   220|
|                Help|                   192|
|             Upgrade|                   168|
|               Error|                   117|
|      Submit Upgrade|            

In [ ]:
# Avg time from cancellation to confirmation, compared to avg time on platform
# Avg number of sessions of cancel to other
# Avg length per session of cancel to other
# Avg cummulative time in session compared to other
# Change of Items per Session for cancelers and non cancelers

In [124]:

#spark.sql("SELECT userId FROM data_cleaned WHERE page = 'Cancellation Confirmation'").show()
spark.sql("SELECT userId, COUNT(userId) FROM data_cleaned WHERE userId IN \
                        (SELECT userId FROM data_cleaned WHERE page = 'Cancellation Confirmation') \
                        GROUP BY userId").show()

+------+-------------+
|userId|count(userId)|
+------+-------------+
|   125|           11|
|    51|         2464|
|    54|         3437|
|100014|          310|
|   101|         2149|
|100021|          319|
|    29|         3603|
|    73|          457|
|    87|         1003|
|     3|          254|
|    28|         1781|
|100022|          964|
|100025|          623|
|300007|          143|
|100006|           44|
|    18|          513|
|    70|         1775|
|100005|          216|
|    17|         1102|
|100007|          520|
+------+-------------+
only showing top 20 rows



In [166]:
# Comparison difference between last time stamp ts and time of registration for users how canceled users and non canceled users
canceled_users = np.array(spark.sql("SELECT userId, max(ts)-max(registration) FROM data_cleaned WHERE userId IN \
                        (SELECT userId FROM data_cleaned WHERE page = 'Cancellation Confirmation') \
                        GROUP BY userId").collect()).astype(int)
non_canceled_users = np.array(spark.sql("SELECT userId, max(ts)-max(registration) FROM data_cleaned WHERE userId IN \
                        (SELECT userId FROM data_cleaned WHERE page != 'Cancellation Confirmation') \
                        GROUP BY userId").collect()).astype(int)

dum = np.array(spark.sql("SELECT userId, max(ts),max(registration) FROM data_cleaned WHERE userId IN \
                        (SELECT userId FROM data_cleaned WHERE page = 'Cancellation Confirmation') \
                        GROUP BY userId").collect()).astype(int)

fig = go.Figure()
fig.add_trace(go.Box(y=canceled_users[:,1]/1000/3600/24, name = 'canceling users'))  
fig.add_trace(go.Box(y=non_canceled_users[:,1]/1000/3600/24, name = 'non canceling users')) 
fig.update_yaxes(title_text='Time difference [days]')
fig.update_layout(title_text="Difference between time of registration and last time stamp",
                  template='plotly_white')
fig.show()    
                  


In [165]:
# Distribution of gender of canceling and non canceling users
canceled_users = np.array(spark.sql("SELECT gender, COUNT(DISTINCT(userId)) FROM data_cleaned WHERE userId IN \
                        (SELECT userId FROM data_cleaned WHERE page = 'Cancellation Confirmation') \
                        GROUP BY gender").collect())
non_canceled_users = np.array(spark.sql("SELECT gender, COUNT(DISTINCT(userId)) FROM data_cleaned WHERE userId IN \
                        (SELECT userId FROM data_cleaned WHERE page != 'Cancellation Confirmation') \
                        GROUP BY gender").collect())

from plotly.subplots import make_subplots

fig = make_subplots(rows=1, cols=2,
             subplot_titles=("Canceling users", "Non canceling users"),
             specs=[[{"type": "domain"}, {"type": "domain"}]])

fig.add_trace(
    go.Pie(values=canceled_users[:,1], labels=canceled_users[:,0], name='Canceling users'),
    row=1, col=1
)

fig.add_trace(
    go.Pie(values=non_canceled_users[:,1], labels=non_canceled_users[:,0], name='Canceling users'),
    row=1, col=2
)

fig.update_layout(title_text="Comparison of gender fraction",
                  template='plotly_white')
fig.show()

In [187]:
# Ratio betwenn up and down votes
canceled_users_upvotes = np.array(spark.sql("SELECT userId, COUNT(page) FROM data_cleaned WHERE page = 'Thumbs Up' AND userId IN \
                        (SELECT userId FROM data_cleaned WHERE page = 'Cancellation Confirmation') \
                        GROUP BY userId ORDER BY userId").collect()).astype(int)
canceled_users_downvotes = np.array(spark.sql("SELECT userId, COUNT(page) FROM data_cleaned WHERE page = 'Thumbs Down' AND userId IN \
                        (SELECT userId FROM data_cleaned WHERE page = 'Cancellation Confirmation') \
                        GROUP BY userId ORDER BY userId").collect()).astype(int)
print(canceled_users_downvotes)
print(canceled_users_upvotes)

[[100001      2]
 [100005      3]
 [100006      2]
 [100007      6]
 [100009      8]
 [100011      1]
 [100012      9]
 [100013     15]
 [100014      3]
 [100015      8]
 [100017      1]
 [100019      1]
 [100021      5]
 [100022     20]
 [100023      2]
 [100025      7]
 [   101     16]
 [   103      9]
 [   105      6]
 [   106      6]
 [    12      9]
 [   121      8]
 [   122      1]
 [   129      7]
 [    17     13]
 [    18      1]
 [200001      1]
 [200011     24]
 [200015     10]
 [200016      5]
 [200017     16]
 [200018     11]
 [200020     36]
 [200021     35]
 [200024     13]
 [    28     22]
 [    29     22]
 [     3      3]
 [300001     17]
 [300007      1]
 [    32      1]
 [    51     21]
 [    53     16]
 [    54     29]
 [    58     20]
 [    70     12]
 [    73      7]
 [    87      5]]
[[100001      8]
 [100003      3]
 [100005      7]
 [100006      2]
 [100007     19]
 [100009     23]
 [100012     18]
 [100013     39]
 [100014     17]
 [100015     35]
 [100017     

In [197]:
spark.sql("""
    SELECT userId,
           SUM(CASE WHEN page = 'Thumbs Up' THEN 1 ELSE 0 END)/
           SUM(CASE WHEN page = 'Thumbs Down' THEN 1 ELSE 0 END) AS Ratio_TU_v_TD
    FROM data_cleaned
    WHERE userId IN (SELECT userId FROM data_cleaned WHERE page = 'Cancellation Confirmation')
    GROUP BY userId
""").show()
spark.sql("""
    SELECT userId,
           SUM(CASE WHEN page = 'Thumbs Up' THEN 1 ELSE 0 END)/
           SUM(CASE WHEN page = 'Thumbs Down' THEN 1 ELSE 0 END) AS Ratio_TU_v_TD
    FROM data_cleaned
    WHERE userId IN (SELECT userId FROM data_cleaned WHERE page != 'Cancellation Confirmation')
    GROUP BY userId
""").show()


+------+------------------+
|userId|     Ratio_TU_v_TD|
+------+------------------+
|   125|              null|
|    51| 4.761904761904762|
|    54| 5.620689655172414|
|100014| 5.666666666666667|
|   101|             5.375|
|100021|               2.2|
|    29|               7.0|
|    73|               2.0|
|    87|               6.4|
|     3| 4.666666666666667|
|    28| 3.590909090909091|
|100022|              2.25|
|100025| 2.857142857142857|
|300007|              11.0|
|100006|               1.0|
|    18|              20.0|
|    70|               7.5|
|100005|2.3333333333333335|
|    17| 3.076923076923077|
|100007|3.1666666666666665|
+------+------------------+
only showing top 20 rows

+------+------------------+
|userId|     Ratio_TU_v_TD|
+------+------------------+
|100010|               3.4|
|200002|               3.5|
|   125|              null|
|   124| 4.170731707317073|
|    51| 4.761904761904762|
|     7|               7.0|
|    15| 5.785714285714286|
|    54| 5.62068965517

In [209]:
spark.sql("SELECT userId, max(ts)-max(registration) FROM data_cleaned WHERE userId IN \
                        (SELECT userId FROM data_cleaned WHERE page = 'Cancellation Confirmation') \
                        GROUP BY userId").show()
spark.sql("SELECT userId, max(ts)-max(registration) FROM data_cleaned WHERE userId NOT IN \
                        (SELECT userId FROM data_cleaned WHERE page = 'Cancellation Confirmation') \
                        GROUP BY userId").show()

+------+-----------------------------+
|userId|(max(ts) - max(registration))|
+------+-----------------------------+
|   125|                   6161779000|
|    51|                   1680985000|
|    54|                   9568946000|
|100014|                   7351206000|
|   101|                   4662657000|
|100021|                   5593438000|
|    29|                   5192990000|
|    73|                   4299444000|
|    87|                   5108474000|
|     3|                   6989352000|
|    28|                   3828767000|
|100022|                   4140310000|
|100025|                   7673583000|
|300007|                    998240000|
|100006|                    788587000|
|    18|                   3320524000|
|    70|                  12591764000|
|100005|                   7360899000|
|    17|                   1131755000|
|100007|                   9969490000|
+------+-----------------------------+
only showing top 20 rows

+------+-----------------------------+

In [201]:
spark.sql("SELECT userId, count(userId) FROM data_cleaned WHERE page != 'Cancellation Confirmation' GROUP BY userId").show()

+------+-------------+
|userId|count(userId)|
+------+-------------+
|100010|          381|
|200002|          474|
|   125|           10|
|    51|         2463|
|   124|         4825|
|     7|          201|
|    54|         3436|
|    15|         2278|
|   155|         1002|
|   132|         2304|
|   154|          118|
|100014|          309|
|   101|         2148|
|    11|          848|
|   138|         2469|
|300017|         4428|
|    29|         3602|
|    69|         1342|
|100021|          318|
|    42|         4257|
+------+-------------+
only showing top 20 rows



In [208]:
len(np.array(spark.sql("SELECT userId, count(userId) FROM data_cleaned WHERE page = 'Cancellation Confirmation' GROUP BY userId").collect()))

52

In [125]:
spark.sql("SELECT userId, COUNT(userId) FROM data_cleaned WHERE userId IN \
                        (SELECT userId FROM data_cleaned WHERE page != 'Cancellation Confirmation') \
                        GROUP BY userId").show()

+------+-------------+
|userId|count(userId)|
+------+-------------+
|100010|          381|
|200002|          474|
|   125|           11|
|   124|         4825|
|    51|         2464|
|     7|          201|
|    15|         2278|
|    54|         3437|
|   155|         1002|
|100014|          310|
|   132|         2304|
|   154|          118|
|   101|         2149|
|    11|          848|
|   138|         2469|
|300017|         4428|
|100021|          319|
|    29|         3603|
|    69|         1342|
|   112|          292|
+------+-------------+
only showing top 20 rows



In [120]:
spark.sql("SELECT userId, COUNT(userId) FROM data_cleaned GROUP BY userId").show()

+------+-------------+
|userId|count(userId)|
+------+-------------+
|100010|          381|
|200002|          474|
|   125|           11|
|    51|         2464|
|   124|         4825|
|     7|          201|
|    54|         3437|
|    15|         2278|
|   155|         1002|
|   132|         2304|
|   154|          118|
|100014|          310|
|   101|         2149|
|    11|          848|
|   138|         2469|
|300017|         4428|
|    29|         3603|
|    69|         1342|
|100021|          319|
|    42|         4257|
+------+-------------+
only showing top 20 rows



In [103]:
spark.sql("SELECT * FROM data_cleaned WHERE sessionId IN \
         (SELECT sessionId - 1 FROM data_cleaned WHERE page = 'Cancellation Confirmation')").show()

+--------------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+---------+-------------+---------+--------------------+------+-------------+--------------------+------+
|              artist|     auth|firstName|gender|itemInSession|lastName|   length|level|            location|method|     page| registration|sessionId|                song|status|           ts|           userAgent|userId|
+--------------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+---------+-------------+---------+--------------------+------+-------------+--------------------+------+
|                null|Logged In|  Ainsley|     F|            0|  Farley|     null| free|McAllen-Edinburg-...|   GET|     Home|1538304455000|      187|                null|   200|1538359181000|"Mozilla/5.0 (Win...|    78|
|       Skunk Anansie|Logged In|  Ainsley|     F|            1|  Farley|194.58567| free|McAllen-Edinburg-...|   PUT|

In [217]:
data_cleaned.show()

+--------------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+---------------+-------------+---------+--------------------+------+-------------+--------------------+------+
|              artist|     auth|firstName|gender|itemInSession|lastName|   length|level|            location|method|           page| registration|sessionId|                song|status|           ts|           userAgent|userId|
+--------------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+---------------+-------------+---------+--------------------+------+-------------+--------------------+------+
|      Martha Tilston|Logged In|    Colin|     M|           50| Freeman|277.89016| paid|     Bakersfield, CA|   PUT|       NextSong|1538173362000|       29|           Rockpools|   200|1538352117000|Mozilla/5.0 (Wind...|    30|
|    Five Iron Frenzy|Logged In|    Micah|     M|           79|    Long|236.09424| free|Bost

# Exploratory Data Analysis
When you're working with the full dataset, perform EDA by loading a small subset of the data and doing basic manipulations within Spark. In this workspace, you are already provided a small subset of data you can explore.

### Define Churn

Once you've done some preliminary analysis, create a column `Churn` to use as the label for your model. I suggest using the `Cancellation Confirmation` events to define your churn, which happen for both paid and free users. As a bonus task, you can also look into the `Downgrade` events.

### Explore Data
Once you've defined churn, perform some exploratory data analysis to observe the behavior for users who stayed vs users who churned. You can start by exploring aggregates on these two groups of users, observing how much of a specific action they experienced per a certain time unit or number of songs played.

In [51]:
data_feat_eng[data_feat_eng['userId']==124].mean('itemInSession').show()

AttributeError: 'DataFrame' object has no attribute 'mean'

In [42]:
data_feat_eng[data_feat_eng['page']=='Error'].groupBy('userId').count().show()

+------+-----+
|userId|count|
+------+-----+
|   124|    6|
|    51|    1|
|     7|    1|
|    15|    2|
|    54|    1|
|   155|    3|
|   132|    3|
|   101|    3|
|    11|    1|
|   138|    1|
|300017|    5|
|    69|    4|
|100021|    2|
|    42|    7|
|    73|    1|
|200010|    1|
|    30|    1|
|   113|    3|
|    59|    4|
|   146|    1|
+------+-----+
only showing top 20 rows



In [35]:
# Add Churn column
def valueToCategory(value):
   if   value == 'Cancellation Confirmation': return 1
   else : return 0
# Calculate time between registration and timestamp
def timeDifference(registration,ts):
    return float(ts)-float(registration)
# Calculate number of interactions with error page for users
def errorPageInteractions(userId):
    return data_feat_eng[(data_feat_eng['page']=='Error') & (data_feat_eng['userId']==userId)].count()
# Calculate avg. number of Session items
# Calculate avg. session length

from pyspark.sql.functions import udf
from pyspark.sql.types import *

udfValueToCategory = udf(valueToCategory, StringType())
timeDifference = udf(timeDifference, FloatType())
data_feat_eng = data_cleaned.withColumn("Churn", udfValueToCategory('page'))
data_feat_eng = data_cleaned.withColumn("TimeDifference", timeDifference('registration','ts'))
data_feat_eng.show()

+--------------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+---------------+-------------+---------+--------------------+------+-------------+--------------------+------+--------------+
|              artist|     auth|firstName|gender|itemInSession|lastName|   length|level|            location|method|           page| registration|sessionId|                song|status|           ts|           userAgent|userId|TimeDifference|
+--------------------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+---------------+-------------+---------+--------------------+------+-------------+--------------------+------+--------------+
|      Martha Tilston|Logged In|    Colin|     M|           50| Freeman|277.89016| paid|     Bakersfield, CA|   PUT|       NextSong|1538173362000|       29|           Rockpools|   200|1538352117000|Mozilla/5.0 (Wind...|    30|  1.78755008E8|
|    Five Iron Frenzy|Logged In|

# Feature Engineering
Once you've familiarized yourself with the data, build out the features you find promising to train your model on. To work with the full dataset, you can follow the following steps.
- Write a script to extract the necessary features from the smaller subset of data
- Ensure that your script is scalable, using the best practices discussed in Lesson 3
- Try your script on the full data set, debugging your script if necessary

If you are working in the classroom workspace, you can just extract features based on the small subset of data contained here. Be sure to transfer over this work to the larger dataset when you work on your Spark cluster.

# Modeling
Split the full dataset into train, test, and validation sets. Test out several of the machine learning methods you learned. Evaluate the accuracy of the various models, tuning parameters as necessary. Determine your winning model based on test accuracy and report results on the validation set. Since the churned users are a fairly small subset, I suggest using F1 score as the metric to optimize.

# Final Steps
Clean up your code, adding comments and renaming variables to make the code easier to read and maintain. Refer to the Spark Project Overview page and Data Scientist Capstone Project Rubric to make sure you are including all components of the capstone project and meet all expectations. Remember, this includes thorough documentation in a README file in a Github repository, as well as a web app or blog post.